# Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

In [2]:
# Import the API key.
from config import g_key

In [3]:
# Files to load
weather_data_to_load = "data/WeatherPy_challenge.csv"

In [4]:
# Read the data file, and store it in a DataFrame.
vacation_data_df = pd.read_csv(weather_data_to_load)
vacation_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,Ambanja,MG,2020-04-03 17:08:01,-13.68,48.45,79.16,76,40,0.87,light rain,0.066142,0.0
1,Qingquan,CN,2020-04-03 17:08:02,30.45,115.26,56.77,77,94,2.53,overcast clouds,0.000000,0.0
2,Mataura,NZ,2020-04-03 17:08:02,-46.19,168.86,46.99,96,99,2.30,overcast clouds,0.000000,0.0
3,Mindelo,CV,2020-04-03 17:08:02,16.89,-24.98,82.40,51,20,21.92,few clouds,0.000000,0.0
4,Conceicao Do Araguaia,BR,2020-04-03 17:08:02,-8.26,-49.26,91.40,59,75,3.36,broken clouds,0.000000,0.0
5,Ushuaia,AR,2020-04-03 17:04:59,-54.80,-68.30,47.97,51,43,8.25,light rain,0.004331,0.0
6,Ulaanbaatar,MN,2020-04-03 17:08:03,47.91,106.88,23.00,57,0,2.24,clear sky,0.000000,0.0
7,Enkoping,SE,2020-04-03 17:08:03,59.64,17.08,42.80,26,63,17.22,broken clouds,0.000000,0.0
8,Hermanus,ZA,2020-04-03 17:08:03,-34.42,19.23,73.00,83,64,6.76,broken clouds,0.000000,0.0
9,Constitucion,CL,2020-04-03 17:08:03,-35.33,-72.42,70.36,38,24,12.68,few clouds,0.000000,0.0


In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
# Ask the customer to determine if they want it to be raining.
rain_preference = input("Do you want it to be raining? (yes/no) ")
# Ask the customer to determine if they want it to be snowing.
snow_preference = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip?  70
What is the maximum temperature you would like for your trip?  90
Do you want it to be raining? (yes/no)  no
Do you want it to be snowing? (yes/no)  no


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_vacation_df = vacation_data_df.loc[(vacation_data_df["Max Temp"] <= max_temp) & \
                                             (vacation_data_df["Max Temp"] >= min_temp)]

if rain_preference == 'yes':
    preferred_vacation_df = preferred_vacation_df.loc[(preferred_vacation_df["Rain inches (last 3 hrs)"] >0)]
else:
    preferred_vacation_df = preferred_vacation_df.loc[(preferred_vacation_df["Rain inches (last 3 hrs)"] ==0)]

if snow_preference == 'yes':
    preferred_vacation_df = preferred_vacation_df.loc[(preferred_vacation_df["Snow inches (last 3 hrs)"] >0)]
else:
    preferred_vacation_df = preferred_vacation_df.loc[(preferred_vacation_df["Snow inches (last 3 hrs)"] ==0)]

    
preferred_vacation_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
3,Mindelo,CV,2020-04-03 17:08:02,16.89,-24.98,82.40,51,20,21.92,few clouds,0.0,0.0
8,Hermanus,ZA,2020-04-03 17:08:03,-34.42,19.23,73.00,83,64,6.76,broken clouds,0.0,0.0
9,Constitucion,CL,2020-04-03 17:08:03,-35.33,-72.42,70.36,38,24,12.68,few clouds,0.0,0.0
10,Georgetown,MY,2020-04-03 17:08:03,5.41,100.34,82.40,88,20,2.24,few clouds,0.0,0.0
11,Ardakan,IR,2020-04-03 17:08:04,32.31,54.02,73.40,18,48,13.87,scattered clouds,0.0,0.0
20,Barahona,DO,2020-04-03 17:08:05,18.21,-71.10,84.20,58,40,9.17,scattered clouds,0.0,0.0
31,Inhambane,MZ,2020-04-03 17:08:07,-23.86,35.38,75.20,83,75,12.75,drizzle,0.0,0.0
36,Miri,MY,2020-04-03 17:08:08,4.41,114.01,80.60,94,75,8.05,broken clouds,0.0,0.0
38,Arraial Do Cabo,BR,2020-04-03 17:08:09,-22.97,-42.03,86.00,66,33,16.11,scattered clouds,0.0,0.0
39,Carnarvon,AU,2020-04-03 17:08:09,-24.87,113.63,73.40,78,100,14.99,overcast clouds,0.0,0.0


In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, coordinates, and current weather.
hotel_df = preferred_vacation_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
3,Mindelo,CV,82.40,16.89,-24.98,few clouds,
8,Hermanus,ZA,73.00,-34.42,19.23,broken clouds,
9,Constitucion,CL,70.36,-35.33,-72.42,few clouds,
10,Georgetown,MY,82.40,5.41,100.34,few clouds,
11,Ardakan,IR,73.40,32.31,54.02,scattered clouds,
20,Barahona,DO,84.20,18.21,-71.10,scattered clouds,
31,Inhambane,MZ,75.20,-23.86,35.38,drizzle,
36,Miri,MY,80.60,4.41,114.01,broken clouds,
38,Arraial Do Cabo,BR,86.00,-22.97,-42.03,scattered clouds,
39,Carnarvon,AU,73.40,-24.87,113.63,overcast clouds,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
        # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        pass

hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
3,Mindelo,CV,82.40,16.89,-24.98,few clouds,Hotel Gaudi
8,Hermanus,ZA,73.00,-34.42,19.23,broken clouds,Misty Waves Boutique Hotel
9,Constitucion,CL,70.36,-35.33,-72.42,few clouds,Hotel Boutique Las Azucenas
10,Georgetown,MY,82.40,5.41,100.34,few clouds,Cititel Penang
11,Ardakan,IR,73.40,32.31,54.02,scattered clouds,Ardakan Hotel
...,...,...,...,...,...,...,...
545,Innisfail,AU,70.47,-17.53,146.03,clear sky,Moondarra Motel
547,Saint-Georges,GD,84.20,12.06,-61.75,few clouds,Radisson Grenada Beach Resort
550,Salina Cruz,MX,89.26,16.17,-95.20,clear sky,Hotel Amerik
557,Vao,NC,76.53,-22.67,167.48,overcast clouds,Hotel Kou-Bugny


In [18]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt>
<dd>{Hotel Name}</dd>
<dt>City</dt>
<dd>{City}</dd>
<dt>Country</dt>
<dd>{Country}</dd>
<dt>Current Weather</dt>
<dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [20]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a marker layer map for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Create the output file (CSV).
output_challenge_data_file = "data/WeatherPy_vacation.csv"

# Export the hotel dataframe into a CSV.
hotel_df.to_csv(output_challenge_data_file, index=False)